# 🚀 Your First AI Agent: From Prompt to Action

- ✅ Install [Agent Development Kit (ADK)](https://google.github.io/adk-docs/)
- ✅ Configure your API key to use the Gemini model
- ✅ Build your first simple agent
- ✅ Run your agent and watch it use a tool (like Google Search) to answer a question

# 1. Setup

This notebook uses the [Gemini API](https://ai.google.dev/gemini-api/docs), which requires authentication.

In [1]:
# API key
import os
from dotenv import load_dotenv

try:
    load_dotenv()
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

# Import components
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")

# Url for web UI
ADK_URL = "http://localhost:8000"
print(f"✅ ADK Web UI will be available at: {ADK_URL}")

✅ Gemini API key setup complete.
✅ ADK components imported successfully.
✅ ADK Web UI will be available at: http://localhost:8000


---

# 2: AI Agent with ADK

### 2.1 Define Agent

In [2]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

root_agent = Agent(
    name="helpful_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)

print("✅ Root Agent defined.")

✅ Root Agent defined.


## 2.2 Run agent

Now it's time to bring agent to life and send it a query. To do this, you need a [`Runner`](https://google.github.io/adk-docs/runtime/), which is the central component within ADK that acts as the orchestrator. It manages the conversation, sends our messages to the agent, and handles its responses.

**a. Create an `InMemoryRunner` and tell it to use our `root_agent`:**

In [3]:
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner created.")

✅ Runner created.


👉 Note that we are using the Python Runner directly in this notebook. You can also run agents using ADK command-line tools such as `adk run`, `adk web`, or `adk api_server`. To learn more, check out the documentation related to [runtime in ADK](https://google.github.io/adk-docs/runtime/).

**b. Now you can call the `.run_debug()` method to send our prompt and get an answer.**

👉 This method abstracts the process of session creation and maintenance and is used in prototyping. We'll explore "what sessions are and how to create them" on Day 3.

In [4]:
response = await runner.run_debug("What's the weather in London?")


 ### Created new session: debug_session_id

User > What's the weather in London?
helpful_assistant > Currently in London, it is cloudy with a temperature of 44°F (7°C). The humidity is around 97%, and there is a 0% chance of rain. The feel-like temperature is 41°F (5°C).

Looking ahead, the forecast for today, Sunday, January 18, 2026, predicts cloudy weather during the day and mostly cloudy at night, with a 20% chance of rain. Temperatures are expected to range between 41°F (5°C) and 51°F (11°C).

The weather for the next few days includes:
*   **Monday, January 19:** Light rain throughout the day and night, with temperatures between 44°F (7°C) and 50°F (10°C).
*   **Tuesday, January 20:** Light rain during the day and night, with temperatures between 41°F (5°C) and 50°F (10°C).
*   **Wednesday, January 21:** Light rain during the day and night, with temperatures between 41°F (5°C) and 47°F (8°C).


---
# 3: ADK Web Interface

### Overview

ADK includes a built-in web interface for interactively chatting with, testing, and debugging your agents.

<img width="1200" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/adk-web-ui.gif" alt="ADK Web UI" />

To use the ADK web UI, you'll need to create an agent with Python files using the `adk create` command.

Run the command below to generate a `sample-agent` folder that contains all the necessary files, including `agent.py` for your code, an `.env` file with your API key pre-configured, and an `__init__.py` file:

In [6]:
if not os.path.isdir('sample_agent'):
    !adk create sample_agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY

Now we can run ADK web:

In [7]:
# For local development, just print the URL
print(f"Open ADK Web UI at: {ADK_URL}")

# Run ADK web UI locally (access at http://localhost:8000)
!adk web

Open ADK Web UI at: http://localhost:8000
2026-01-17 23:40:27,188 - INFO - service_factory.py:220 - Using in-memory memory service
2026-01-17 23:40:27,188 - INFO - local_storage.py:83 - Using per-agent session storage rooted at /Users/xing.zhang/github/google_5day_ai_202511
2026-01-17 23:40:27,188 - INFO - local_storage.py:109 - Using file artifact service at /Users/xing.zhang/github/google_5day_ai_202511/.adk/artifacts
/Users/xing.zhang/anaconda3/envs/google-adk/lib/python3.11/site-packages/google/adk/cli/fast_api.py:141: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  credential_service = InMemoryCredentialService()
/Users/xing.zhang/anaconda3/envs/google-adk/lib/python3.11/site-packages/google/adk/auth/credential_service/in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental a

Now you can access the ADK dev UI using the link above.

Once you open the link, you'll see the ADK web interface where you can ask your ADK agent questions.

Note: This sample agent does not have any tools enabled (like Google Search). It is a basic agent designed specifically to let you explore the UI features.

---
# 4. xing's additional notes - connect to local db to check saved sessions

In [8]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('sample_agent/.adk/session.db')
cursor = conn.cursor()

# List all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tables:", cursor.fetchall())

# View sessions
cursor.execute("SELECT * FROM events;")
for row in cursor.fetchall():
    print(row)

conn.close()

Tables: [('app_states',), ('user_states',), ('sessions',), ('events',)]
('e7df0eed-6a37-43e1-bb47-7b6b53c450da', 'sample_agent', 'user', 'b629e8c9-a8ab-4136-a38e-957cf11070b7', 'e-81560f03-7ba7-42bc-b80e-2e8f2144a8bf', 1768201030.413868, '{"content":{"parts":[{"text":"hello"}],"role":"user"},"invocation_id":"e-81560f03-7ba7-42bc-b80e-2e8f2144a8bf","author":"user","actions":{"state_delta":{},"artifact_delta":{},"requested_auth_configs":{},"requested_tool_confirmations":{}},"id":"e7df0eed-6a37-43e1-bb47-7b6b53c450da","timestamp":1768201030.413868}')
('04b92ea2-ca23-4dab-96e4-31b849f4786c', 'sample_agent', 'user', 'b629e8c9-a8ab-4136-a38e-957cf11070b7', 'e-81560f03-7ba7-42bc-b80e-2e8f2144a8bf', 1768201030.416185, '{"model_version":"gemini-2.5-flash-lite","content":{"parts":[{"text":"Hello! How can I help you today?"}],"role":"model"},"finish_reason":"STOP","usage_metadata":{"candidates_token_count":9,"prompt_token_count":40,"prompt_tokens_details":[{"modality":"TEXT","token_count":40}],"t